In [ ]:
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 77.7 MB/s eta 0:00

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.315
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [ ]:
from google.colab import drive

drive.mount('/content/drive/')


Mounted at /content/drive/


In [ ]:
!tar -xzf /content/drive/MyDrive/transcripts/podcasts-transcripts-0to2.tar.gz -C /content/drive/MyDrive/transcripts/

tar (child): /content/drive/MyDrive/transcripts/podcasts-transcripts-0to2.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
folder = "/content/drive/MyDrive/0"

In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader(folder, glob='**/*.json', show_progress=True, loader_cls=TextLoader)

documents = loader.load()


100%|██████████| 130/130 [00:14<00:00,  9.27it/s]


In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = '/content/drive/MyDrive/db'

## the type of embedding used
embedding = instructor_embeddings

#loading into chroma
vectordb = Chroma.from_documents(documents=documents,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [ ]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("What do you pick nurse practitioner nursing informatics nurse anesthesia, or maybe go to med school")

In [ ]:
len(docs)

4

In [ ]:
docs[0]


Document(page_content='{"results": [{"alternatives": [{"transcript": "I\'m not kidding.", "confidence": 0.8443856835365295, "words": [{"startTime": "5.700s", "endTime": "5.800s", "word": "I\'m"}, {"startTime": "5.800s", "endTime": "6.300s", "word": "not"}, {"startTime": "6.300s", "endTime": "6.800s", "word": "kidding."}]}]}, {"alternatives": [{"transcript": "Is it to feel like y\'all for my follow-up strategy to fix two to level two solutions to this comprehend?", "confidence": 0.7447347044944763, "words": [{"startTime": "32.900s", "endTime": "33s", "word": "Is"}, {"startTime": "33s", "endTime": "33.200s", "word": "it"}, {"startTime": "33.200s", "endTime": "33.300s", "word": "to"}, {"startTime": "33.300s", "endTime": "33.500s", "word": "feel"}, {"startTime": "33.500s", "endTime": "33.700s", "word": "like"}, {"startTime": "33.700s", "endTime": "34s", "word": "y\'all"}, {"startTime": "34s", "endTime": "34.200s", "word": "for"}, {"startTime": "34.200s", "endTime": "34.400s", "word": "my"}